In [5]:
n=3

In [7]:
G = SymmetricGroup(n)

In [7]:
SGA = G.algebra(QQ)

In [8]:
A = SGA.dft(); A

[   1    1    1    1    1    1]
[   1 -1/2 -1/2  1/2  1/2   -1]
[   0 -3/4  3/4  3/4 -3/4    0]
[   0    1   -1    1   -1    0]
[   1 -1/2 -1/2 -1/2 -1/2    1]
[   1    1    1   -1   -1   -1]

In [93]:
#is the matrix unitary?
#A*A^T, the Gram matrix, is diagonal, but not the identity. so A is not unitary.

In [9]:
print((A*A.transpose()).str())

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]


In [1]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick: 
#define a new inner product <x,y>_\rho = (1/|G|)\sum_{g \in G}<\rho(g)x,\rho(g)y>
#compute an orthonormal basis using this inner product

In [210]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition):
    n = sum(partition)
    G = SymmetricGroup(n)
    specht_module = G.algebra(QQ).specht_module(partition)
    d_part = specht_module.dimension()
    return sqrt(d_part/G.order())*sum(f(g)*specht_module.representation_matrix(g) for g in G)

In [11]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
def delta(s):
    return lambda t: 1 if t == s else 0

In [12]:
#for each basis element g \in G
#compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
#make the Fourier coefficients into a list via .list()
#flatten the list of lists to get a vector \hat{g}
#put the vectors into a list and take the transpose to get the DFT
from sage.misc.flatten import flatten
def unitary_dft(n):
    return matrix([flatten([hat(delta(g),partition).list() for partition in Partitions(n)]) for g in G]).transpose()

In [257]:
#define the standard inner product on k[S_n]
def std_inner_product(n,x,y):
    G = SymmetricGroup(n)
    return sum(x[g]*conjugate(y[g]) for g in G)

In [10]:
#define the G-averaged inner product given a representation
#<x,y>_\rho = (1/|G|)\sum_{g \in G}<\rho(g)x,\rho(g)y>
def inv_inner_product(partition,x,y):
    n = sum(partition)
    G = SymmetricGroup(n)
    rho = G.algebra(QQ).specht_module(partition).representation_matrix
    return (1/G.order())*sum((rho(g)*vector(x)).inner_product(rho(g)*vector(y)) for g in G)

In [195]:
#define the projection of the vector v onto vector u
def proj(u,v):
    return (vector(v).inner_product(vector(u))/vector(u).inner_product(vector(u)))*vector(u)

In [231]:
#define Gram-Schmidt orthonormalization process that works in the symbolic ring
#assume A = [v_1, v_2, ... , v_k] and each v_j is a column vector
#NOTE: GS, M = A.gram_schmidt(orthonormal=True) works for some rings, but not \QQ or exact rings
def gram_schmidt_SR(A,orthonormal=False):
    v = A.transpose()
    u = []
    if orthonormal:
        e = []
    for k in range(A.ncols()):
        u_k = v[k] - sum(proj(u[j],v[k]) for j in range(k))
        u.append(u_k)
        if orthonormal:
            e.append(u_k/u_k.norm())
    if orthonormal:
        return matrix(e).transpose()
    else:
        return matrix(u).transpose()

In [232]:
#find the change-of-basis matrix M for which A = M*GS where GS is the Gram-Schmidt orthonormal basis of A
def unitary_change_of_basis(A):
    orthonormal_vecs = gram_schmidt_SR(A,orthonormal=True)
    return (A*orthonormal_vecs.inverse()).simplify_full()

In [26]:
#define the inner product matrix A = [<e_i,e_j>_\rho] for some ordered basis of {e_i} of V_\rho
#ISSUE: is the inner product on V_\rho really defined by making the basis vectors orthonormal?
#or is it just the inner product on k[S_n] which is k^6, which restricts to the invariant subspace?
def inner_product_matrix(partition):
    specht_module = G.algebra(QQ).specht_module(partition)
    d_part = specht_module.dimension()
    e = lambda i: [1 if i==j else 0 for j in range(d_part)]
    return matrix([[inv_inner_product(partition,e(i),e(j)) for j in range(d_part)] for i in range(d_part)])

In [27]:
partition = Partitions(n)[1]
specht_module = G.algebra(QQ).specht_module(partition)
inner_product_matrix(partition)

[ 4/3 -2/3]
[-2/3  4/3]

In [15]:
repn_basis = [b.lift() for b in specht_module.basis()]

In [25]:
list(repn_basis[0])

[((), 1), ((1,2,3), -1), ((2,3), -1), ((1,2), 1)]

In [23]:
vector(repn_basis[0])

(1, 0, -1, -1, 0, 1)

In [267]:
A

[ 4/3 -2/3]
[-2/3  4/3]

In [276]:
#check that the change-of-basis matrix really turns \rho into a unitary representation
G = SymmetricGroup(n)
A = inner_product_matrix(partition); print(A)
M = unitary_change_of_basis(A); print(M)
rho = specht_module.representation_matrix
U = M.inverse()*rho(G[1])*M
print(U.transpose()*U)

[ 4/3 -2/3]
[-2/3  4/3]
[  2/5*sqrt(5) -8/15*sqrt(5)]
[            0   2/3*sqrt(5)]
[  18/5  -39/5]
[ -39/5 773/45]


In [239]:
orthonormal_vecs = gram_schmidt_SR(A,orthonormal=True)

In [243]:
repn_basis = [b.lift() for b in specht_module.basis()]; repn_basis

[() - (2,3) + (1,2) - (1,2,3), -(2,3) - (1,2,3) + (1,3,2) + (1,3)]

In [255]:
repn_basis[0][G[1]]

0

In [260]:
std_inner_product(n,repn_basis[0],repn_basis[1])

2

In [241]:
A == M*orthonormal_vecs().simplify_full()

True

In [177]:
repn_basis = [b.lift() for b in G.algebra(QQ).specht_module(partition).basis()]

In [141]:
unitary_change_of_basis(A).simplify_full()

[ 12/5*sqrt(5) -16/5*sqrt(5)]
[            0     4*sqrt(5)]

In [153]:
unitary_dft(3)*unitary_dft(3).transpose()

[   1    0    0    0    0    0]
[   0  4/3 -2/3  2/3 -1/3    0]
[   0 -2/3  4/3 -1/3  2/3    0]
[   0  2/3 -1/3  4/3 -2/3    0]
[   0 -1/3  2/3 -2/3  4/3    0]
[   0    0    0    0    0    1]

In [58]:
#what are the eigenvalues?
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [92]:
eigs = A.eigenvalues(); eigs

[-1.927382440195288?, 2.121715010353017?, -1.174423554578382? - 1.623003052198150?*I, -1.174423554578382? + 1.623003052198150?*I, 1.452257269499517? - 1.086817272076326?*I, 1.452257269499517? + 1.086817272076326?*I]

In [93]:
[abs(eig) for eig in eigs]

[1.927382440195288?,
 2.121715010353017?,
 2.003349593304431?,
 2.003349593304431?,
 1.813897174510622?,
 1.813897174510622?]

In [121]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))

[2.44948974+0.j 2.44948974+0.j 2.        +0.j 1.73205081+0.j
 1.73205081+0.j 1.5       +0.j]
[2.44948974 1.73205081 1.5        2.         1.73205081 2.44948974]
